In [1]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)

> upcoming_web_scrape func:

    get_upcoming_event_url(xml)
    get_upcoming_event_data(upcoming_event_url)


#### User variables:

In [2]:
output_loc = '../data/'

# Event's list for update:

#### Get last update:

In [3]:
files = get_content(directory=output_loc)
last_file_update = None

for file in files:
    if 'ufc_fighters_stats' in file:
        last_file_update = get_last_update(file_path = output_loc + file)

print('Last update: ' + last_file_update)

Last update: 06-14-2023


In [4]:
#last_file_update = None #for manuel updating

#### Get links of events to add to data:

In [5]:
events = get_links(url='http://ufcstats.com/statistics/events/completed?page=all')
events.sort_values(by='date', ascending=False)

# filter this df to only add new matches
if last_file_update == None:
        events_for_update = events.event_url
        df = events
else:
    df = events[events.date > last_file_update].reset_index(drop=True)
    events_for_update = df.event_url
    
print('Events required update: %d' %len(events_for_update))   
df.tail(3)

Events required update: 649


,date,event_url
646,1995-04-07,http://ufcstats.com/event-details/b60391da771d...
647,1994-12-16,http://ufcstats.com/event-details/1a49e0670dfa...
648,1994-09-09,http://ufcstats.com/event-details/a6a9ab5a824e...


#### Saving file:

In [6]:
df.to_csv(output_loc + 'update_list_events.csv', index=0)
df.tail(3)

,date,event_url
646,1995-04-07,http://ufcstats.com/event-details/b60391da771d...
647,1994-12-16,http://ufcstats.com/event-details/1a49e0670dfa...
648,1994-09-09,http://ufcstats.com/event-details/a6a9ab5a824e...


# Fighter's list for update:

#### Get latest events:

In [7]:
df = pd.read_csv(output_loc + 'update_list_events.csv')
df.tail(3)

,date,event_url
646,1995-04-07,http://ufcstats.com/event-details/b60391da771d...
647,1994-12-16,http://ufcstats.com/event-details/1a49e0670dfa...
648,1994-09-09,http://ufcstats.com/event-details/a6a9ab5a824e...


#### Get list of fighters from even's links:

In [8]:
parts = []

for url in tqdm(df.event_url):
    parts.extend(get_participants(event_link=url))
    
data = pd.DataFrame(data=parts, columns=['fighter','fighter_url'])
data = data.drop_duplicates(subset=['fighter']).reset_index(drop=True)

print('Fighters required update: %d' %len(data))
data.tail(3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649/649 [06:12<00:00,  1.74it/s]

Fighters required update: 2390


,fighter,fighter_url
2387,David Levicki,http://ufcstats.com/fighter-details/49590e0508...
2388,Ray Wizard,http://ufcstats.com/fighter-details/ea0ad15545...
2389,Sean Daugherty,http://ufcstats.com/fighter-details/a683f9ddb7...


#### Saving file:

In [9]:
data.to_csv(output_loc + 'update_list_fighters.csv', index=0)
data.tail(3)

,fighter,fighter_url
2387,David Levicki,http://ufcstats.com/fighter-details/49590e0508...
2388,Ray Wizard,http://ufcstats.com/fighter-details/ea0ad15545...
2389,Sean Daugherty,http://ufcstats.com/fighter-details/a683f9ddb7...
